## Imports

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Set Device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [ ]:
# While loading MNIST, input is 64(N)*1*28*28
# where we can view this as (28*28): 28 time sequences and each sequence has 28 features
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

## Create RNN

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # N * time_seeq * features
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)
        
    def forward(self,x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward Prop
        out, _ = self.rnn(x, h0)    # out and hidden state are parameters, but we do not require 2nd parameter
        out = out.reshape(out.shape[0], -1)     # keep the batch as 1st axis
        out = self.fc(out)
        return out

## Load Data

In [ ]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## Initialize Network

In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

## Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train Network

In [ ]:
for epochs in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #get data to cuda
        data = data.to(device=device).squeeze(1)    # squeeze is added as got error expected 2D, got 3D
        targets = targets.to(device=device)
        
        # No need for flattening to get correct shape
        
        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # Backword
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient descent on adam step
        optimizer.step()

## Check Accuracy

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()


In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)